In [1]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

from typing import List, Tuple

import os
import json
import random

# Constants

In [9]:
START_INDEX = 0

# Data

In [2]:
PATH = "..\\..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

len(data)

83755

In [21]:
CATEGORIES_PATH = ".\\categories.txt"

with open(CATEGORIES_PATH, "r", encoding="UTF-8") as file:
    categories_array = file.read().splitlines()

categories_txt = "\n".join([f"{i}. {category}" for i, category in enumerate(categories_array, start=1)])

# Model

In [11]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "..\\..\\secrets\\t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params, temperature=0)

# Utils

In [34]:
def transform_data(data, **kwargs):
    """
    Преобразует массив словарей, оставляя только нужные ключи.

    :param data: Исходный массив словарей.
    :return: Новый массив словарей с преобразованными данными.
    """
    transformed_data = []
    for item in data:
        new_item = {
            "categories": categories_txt,
            "format_instructions": kwargs["format_instructions"],
            "problem_title": item.get("title", None),
            "problem_categories": item["category"]
        }
        transformed_data.append(new_item)
    return transformed_data

In [5]:
# Функция для создания батчей
def create_batches(array, batch_size, start_index=0):
    """
    Разделяет массив на батчи заданного размера, начиная с указанного индекса.

    :param array: Исходный массив (список).
    :param batch_size: Размер одного батча.
    :param start_index: Индекс, с которого начинать создание батчей.
    :return: Список батчей (списков).
    """
    if batch_size <= 0:
        raise ValueError("Размер батча должен быть больше 0.")
    # Обрезаем массив, начиная с start_index
    array = array[start_index:]
    batches = [array[i:i + batch_size] for i in range(0, len(array), batch_size)]
    return batches

In [6]:
def create_model_prompts(system_prompt: str, user_prompt: str) -> ChatPromptTemplate:
    """
    Создает шаблон промпта для модели.

    :param system_prompt: Системный промпт.
    :param user_prompt: Пользовательский промпт.
    :return: Шаблон промпта.
    """
    system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])
    return chat_prompt

# Prompts

In [8]:
class ClassifyResponse(BaseModel):
    category: str = Field(..., description="""Ответ в формате: КАТЕГОРИЯ""")

parser = PydanticOutputParser(pydantic_object=ClassifyResponse)

SYSTEM_PROMPT = """
Ты — эксперт в классификации товаров. Твоя задача — определить категорию товара на основе его названия и вложенных категорий. Используй следующие категории:

{categories}

Правила классификации:  
- Внимательно анализируй название товара.  
- Если название явно указывает на категорию, выбери соответствующую.  
- Если категория не очевидна, выбери наиболее подходящую из предложенных.
- Если в названии товарной позиции описывается действие, то это должна быть категория "Услуги"
- ПО считается услугой


Правила для формирования ответа:
1. **Финальный ответ должен быть представлен в виде JSON-объекта, заключённого в один блок кода.**  
   - Финальный ответ должен быть корректным JSON-объектом, который заключается в один блок кода (используя три обратных апострофа ```).
   - Внутри JSON-объекта не должно быть дополнительных объяснений или текстовых комментариев.
2. **Фраза "Окончательный ответ" должна предварять блок кода с JSON-объектом.**  
   - Перед блоком кода с JSON-объектом должна быть написана фраза **"Окончательный ответ:"** (без кавычек в самой фразе).  
   - Между фразой и блоком кода не должно быть пустых строк.

Формат ответа должен соответствовать этому: {format_instructions}
""".strip()

USER_PROMPT = """
Товарная позиция: {problem_title}
Вложенные категории: {problem_categories}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

# Transform data

In [41]:
transformed_data = transform_data(data, format_instructions=parser.get_format_instructions())

batches = create_batches(transformed_data, batch_size=100, start_index=START_INDEX)

In [12]:
from os import mkdir

mkdir("dataset")

In [13]:
def get_category(item, parser, categories_array):
    if "Окончательный ответ:" not in item.content:
        return ""
    content_part = item.content.split("Окончательный ответ:")[1]
    parsed = parser.invoke(content_part)
    return parsed.category if parsed.category in categories_array else ""

In [ ]:
chain = prompt | llm

for i, batch in enumerate(batches, start=START_INDEX):
    results = chain.batch(batch) 

    parsed_results = [get_category(item, parser, categories_array) for item in results]

    dump_data = [{
        "title": product["problem_title"],
        "categories": product["problem_categories"],
        "answer": res
    } for product, res in zip(batch, parsed_results)]

    with open(f".\\dataset\\batch_{i}.json", "w", encoding="UTF-8") as file:
        print(f"Save: {file.name}")
        json.dump(dump_data, file, ensure_ascii=False, indent=4)

# Load dataset

In [19]:
import re
from unidecode import unidecode

def transliterate_and_clean(filename):
    """
    Переводит строку в транслит и удаляет запрещенные символы для имени файла.

    :param filename: Исходное имя файла
    :return: Очищенное и транслитерированное имя файла
    """
    # Переводим в транслит
    transliterated = unidecode(filename)
    
    # Удаляем запрещенные символы (оставляем только буквы, цифры, пробелы и дефисы)
    cleaned = re.sub(r'[^\w\s-]', '', transliterated)
    
    # Заменяем пробелы на подчеркивания
    cleaned = cleaned.replace(' ', '_')
    
    return cleaned.strip()

def group_by_answer_category(folder_path, output_folder, correct_categories_folder):
    """
    Группирует элементы по значению answer_category, сортирует их по ключу title,
    сохраняет их в отдельные файлы с транслитерированными и очищенными именами,
    и создает пустые файлы для каждой категории в папке correct_categories.

    :param folder_path: Путь к папке с JSON файлами
    :param output_folder: Путь к папке для сохранения результатов
    :param correct_categories_folder: Путь к папке для пустых файлов с категориями
    """
    # Инициализируем словарь для группировки данных
    grouped_data = {}

    # Проверяем, существует ли указанная папка
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"Папка '{folder_path}' не найдена.")

    # Создаем папки для выходных файлов, если они не существуют
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(correct_categories_folder, exist_ok=True)

    # Перебираем все файлы в папке
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Проверяем, что это файл и он имеет расширение .json
        if os.path.isfile(file_path) and filename.endswith('.json'):
            try:
                # Открываем и читаем файл
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)

                    # Проверяем, что данные являются списком (массивом)
                    if isinstance(data, list):
                        for item in data:
                            # Проверяем, что элемент содержит ключ "answer_category"
                            if "answer" in item:
                                category = item["answer"]

                                # Добавляем элемент в соответствующую категорию
                                if category not in grouped_data:
                                    grouped_data[category] = []
                                grouped_data[category].append(item)
                            else:
                                print(f"Элемент в файле '{filename}' не содержит ключа 'answer_category'. Пропускаем.")
                    else:
                        print(f"Файл '{filename}' содержит данные, которые не являются массивом. Пропускаем.")
            except json.JSONDecodeError:
                print(f"Ошибка при декодировании JSON из файла '{filename}'. Пропускаем.")
            except Exception as e:
                print(f"Неожиданная ошибка при обработке файла '{filename}': {e}")

    # Сортируем данные по ключу title и сохраняем в отдельные файлы
    for category, items in grouped_data.items():
        # Транслитерируем и очищаем название категории
        clean_category_name = transliterate_and_clean(category)

        # Сортировка элементов по ключу title
        sorted_items = sorted(items, key=lambda x: x.get("title", ""))

        # Формируем имя файла и сохраняем данные
        output_file = os.path.join(output_folder, f"{clean_category_name}.json")
        with open(output_file, 'w', encoding='utf-8') as outfile:
            json.dump(sorted_items, outfile, ensure_ascii=False, indent=4)
        print(f"Сохранено {len(sorted_items)} элементов в файл '{output_file}'.")

        # Создаем пустой файл в папке correct_categories
        correct_category_file = os.path.join(correct_categories_folder, f"{clean_category_name}.json")
        open(correct_category_file, 'w').close()  # Создаем пустой файл
        print(f"Создан пустой файл для категории в '{correct_category_file}'.")

In [20]:
# Укажите путь к папке с JSON файлами
dataset_folder = "dataset"

# Укажите путь к папке для сохранения результатов
output_folder = "grouped_data"

# Укажите путь к папке для пустых файлов с категориями
correct_categories_folder = "correct_categories"

# Вызываем функцию для группировки данных
group_by_answer_category(dataset_folder, output_folder, correct_categories_folder)

Сохранено 10619 элементов в файл 'grouped_data\Kantseliarskie_tovary.json'.
Создан пустой файл для категории в 'correct_categories\Kantseliarskie_tovary.json'.
Сохранено 4047 элементов в файл 'grouped_data\Instrumenty_i_oborudovanie.json'.
Создан пустой файл для категории в 'correct_categories\Instrumenty_i_oborudovanie.json'.
Сохранено 2079 элементов в файл 'grouped_data\Instrumenty.json'.
Создан пустой файл для категории в 'correct_categories\Instrumenty.json'.
Сохранено 3723 элементов в файл 'grouped_data\Izdeliia_iz_plastmass.json'.
Создан пустой файл для категории в 'correct_categories\Izdeliia_iz_plastmass.json'.
Сохранено 8058 элементов в файл 'grouped_data\Kompiutery_i_periferiinoe_oborudovanie.json'.
Создан пустой файл для категории в 'correct_categories\Kompiutery_i_periferiinoe_oborudovanie.json'.
Сохранено 586 элементов в файл 'grouped_data\Igry_i_igrushki.json'.
Создан пустой файл для категории в 'correct_categories\Igry_i_igrushki.json'.
Сохранено 796 элементов в файл 'gr